In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
import h5py
from keras.models import load_model

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

In [3]:
train_data_path = '/home/ec2-user/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask/train/'
test_data_path = '/home/ec2-user/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask/test/'

In [8]:
# Read data form training and test 

In [13]:
# X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
#                                              masks_path = train_mask_path, 
#                                              crop_size = 33, 
#                                              num_classes = 6, 
#                                              samples = 100000, 
#                                              balance = True)

In [14]:
# X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
#                                            masks_path = test_mask_path, 
#                                            crop_size = 33, 
#                                            num_classes = 6, 
#                                            samples = 10000, 
#                                            balance = True)

In [15]:
# print(type(X_train), type(Y_train))
# print(X_train.shape, Y_train.shape, num_bands)
# print(X_test.shape,  Y_test.shape,  num_bands)

In [155]:
# X_train_list = np.array([X_train[i] for i in range(X_train.shape[0])])

In [156]:
# X_test_list = np.array([X_test[i] for i in range(X_test.shape[0])])

In [157]:
# Y_train_list = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [16]:
# Y_test_list = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# Batch Generator

This function create a generator for keras.  

In [9]:
def DataGenerator(data_path, masks_path, crop_size, num_classes, samples = 1000, balance = True):  
    while True:
        X, Y, num_bands = utils.load_data(data_path, 
                                          masks_path, 
                                          crop_size, 
                                          num_classes, 
                                          samples,
                                          balance)
        X_for_keras = np.array([X[i] for i in range(X.shape[0])])
        Y_for_keras = np.array([Y[i,:] for i in range(Y.shape[0])])
        yield X_for_keras, Y_for_keras

# Build First Model

In [8]:
# model = Sequential()
# model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1),
#                  activation='relu',
#                  input_shape=(33, 33, 16),
#                  kernel_regularizer=l2(0.01)))

# model.add(MaxPooling2D(pool_size=(2, 2),
#                        strides=(2, 2)))

# model.add(Conv2D(filters=32, kernel_size=(5, 5),
#                  strides=(1, 1),
#                  activation='relu',
#                  kernel_regularizer=l2(0.01)))

# model.add(MaxPooling2D(pool_size=(2, 2),
#                        strides=(2, 2)))
# model.add(Flatten())
# model.add(Dense(128,
#                 activation='relu',
#                 kernel_regularizer=l2(0.01)))
# model.add(Dense(6,
#                 activation='softmax',
#                 kernel_regularizer=l2(0.01)))

In [20]:
model = Sequential()
model.add(Conv2D(filters = 8, kernel_size = (3, 3), strides=(1, 1), input_shape=(17, 17, 16), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, kernel_regularizer=l2(0.01)))
model.add(Activation('softmax'))              

In [21]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [22]:
model.fit_generator(generator = DataGenerator(train_data_path,
                                              train_mask_path, 
                                              crop_size = 17, 
                                              num_classes = 6, 
                                              samples = 5000, 
                                              balance = True),
                    samples_per_epoch = 20,
                    epochs = 10,
                    validation_steps = 1,
                    validation_data = DataGenerator(test_data_path,
                                              test_mask_path, 
                                              crop_size = 17, 
                                              num_classes = 6, 
                                              samples = 5000, 
                                              balance = True))

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=10, validation_steps=1, validation_data=<generator..., steps_per_epoch=20)`
  from ipykernel import kernelapp as app


Epoch 1/10
20/20 [==============================] - 1209s 60s/step - loss: 2.8640 - acc: 0.3796 - val_loss: 2.9791 - val_acc: 0.3269
Epoch 2/10
20/20 [==============================] - 1048s 52s/step - loss: 1.9845 - acc: 0.6648 - val_loss: 2.8267 - val_acc: 0.3526
Epoch 3/10
20/20 [==============================] - 1095s 55s/step - loss: 1.5388 - acc: 0.7941 - val_loss: 2.3922 - val_acc: 0.4708
Epoch 4/10
20/20 [==============================] - 1095s 55s/step - loss: 1.2570 - acc: 0.8538 - val_loss: 2.2135 - val_acc: 0.5858
Epoch 5/10
20/20 [==============================] - 1098s 55s/step - loss: 1.0495 - acc: 0.8872 - val_loss: 2.2895 - val_acc: 0.4735
Epoch 6/10
20/20 [==============================] - 1097s 55s/step - loss: 0.8908 - acc: 0.9071 - val_loss: 3.0908 - val_acc: 0.1878
Epoch 7/10
20/20 [==============================] - 1097s 55s/step - loss: 0.7702 - acc: 0.9183 - val_loss: 4.5198 - val_acc: 0.1677
Epoch 8/10
20/20 [==============================] - 1094s 55s/step - 

In [27]:
model.save('model_1.h5')

In [3]:
model = load_model('model_1.h5')